R Code pulled from https://www.statmethods.net/advgraphs/ggplot2.html

# <span style="color:green">Método BiLi - Hands On</span>

Otim

## <span style="color:cyan">O ciclo INGÊNUO</span>

In [ ]:
library(bibliometrix)
biblioshiny()

In [ ]:
library(remotes)
install_github("elizagrames/litsearchr", ref = "main")
library(litsearchr)
packageVersion("litsearchr")

### <span style="color:pink">Importando os dados</span>
No início busca-se utilizar uma string ingênua de busca para obter um conjunto de artigos relevantes para a pesquisa.

Para limitar a quantidade de artigos, opta-se por restringir o intervalo em 6 a 5 anos do presente ano: Conference papers e articles na língua inglesa.

As buscas podem ser realizadas nas bases de dados WEb of Science, Compendex, IEEE e Scopus

In [ ]:
search_directory <- "/path/with/data.bib"

Ao importar os dados das bases, sugere-se adicionar todas em um único diretório.
E dessa forma, criando um dataset único.

In [ ]:
naiveimport <- litsearchr::import_results(directory = search_directory, verbose = TRUE)

### <span style="color:pink">Remoção dos trabalhos duplicados</span>
Em algumas situações, os dados podem estar duplicados, logo sugere-se a remoção dos mesmos.

In [ ]:
naiveresults <- litsearchr::remove_duplicates(naiveimport, field = "title", method = "string_osa")

Importação das "stop words" pré-definidas e adição de uma lista de "stop words" criadas

In [ ]:
stopworlds <- c("advances","analyse","analysed","analyses",
               "analysing","analysis","analyze","analyzes",
               "analyzed","analyzing", "approach", "assess","assessed",
               "assesses","assessing","assessment","assessments",
               "benefit","based","benefits","change",
               "changed","changes","changing","characteristic",
               "characteristics","characterize","characterized","characterizes",
               "characterizing","clinical","cluster","combine",
               "combined","combines","combining","comorbid",
               "comorbidity","compare","compared","compares",
               "comparing","comparison","conference","control","controlled",
               "controlling","controls", "dataset", "design","designed",
               "designing","effect","effective","effectiveness",
               "effects","efficacy","feasible","feasibility",
               "follow","followed","following","follows",
               "group","groups","impact","intervention",
               "interventions","longitudinal", "method", "moderate","moderated",
               "moderates","moderating","moderator","moderators",
               "outcome","outcomes", "paper", "patient","patients",
               "people","pilot","practice","predict",
               "predicted","predicting","predictor","predictors",
               "predicts","preliminary", "presents", "primary", "proposed", "proposes", "protocol",
               "quality","random","randomise","randomised",
               "randomising","randomize","randomized","randomizing",
               "rationale","reduce","reduced","reduces",
               "reducing","related","report","reported",
               "reporting","reports","response","responses",
               "result","resulted","resulting","results",
               "review","studied","studies","study",
               "studying","systematic","treat","treated",
               "treating","treatment","treatments","treats",
               "trial","trials","versus")

all_stopwords <- c(get_stopwords("English"), stopworlds)

In [ ]:
######### Construção da rede de co-ocorrência de palavras-chave

# Ainda existem alguns termos que não estão relacionados aos outros e ao tópico de interesse. 
# Isso talvez ocorra apenas em um pequeno número de artigos que não mencionam muitos dos 
# outros termos de pesquisa. É necessário uma forma sistemática de identificar esses termos 
# de pesquisa "isolados".

# Uma maneira de fazer isso é analisar os termos de pesquisa como uma rede. 
# A ideia por trás disso é que os termos estão ligados entre si em virtude de aparecerem 
# nos mesmos artigos. Descobrindo quais termos tendem a ocorrer juntos no mesmo 
# artigo, pode-se selecionar grupos de termos que provavelmente se referem ao mesmo tópico 
# e filtrar os termos que não costumam ocorrer junto com qualquer um dos grupos principais 
# de termos.


# Junta todas as keywords obtidas
all_keywords <- unique(append(taggedkeywords, rakedkeywords))


# O título e o resumo de cada artigo serão definidos como o "conteúdo" de cada artigo e 
# conta-se um termo como tendo ocorrido naquele artigo se for encontrado no título ou no 
# resumo. 


# Cria uma ‘document-feature matrix’, uma matriz que registra quais termos aparecem em 
# quais artigos. ‘DFM’ significa ‘matriz de features de documentos’, onde ‘documentos’ 
# são os artigos e ‘features’ são os termos de pesquisa. 

# As linhas da matriz representam os artigos (seus títulos e resumos) e as colunas 
# representam os termos de pesquisa.

naivedfm <-
  litsearchr::create_dfm(
    elements = paste(naiveresults$title, naiveresults$abstract),
    features = all_keywords
  )

# Transforma a matriz em uma rede de termos de pesquisa vinculados para facilitar a 
# visualização das relações entre os termos e a avaliação de sua importância. 

naivegraph <-
  litsearchr::create_network(
    search_dfm = naivedfm,
    min_studies = 10,
    min_occ = 10
  )

# Plotagem da rede
ggraph(naivegraph, layout="stress") +
  coord_fixed() +
  expand_limits(x=c(-3, 3)) +
  geom_edge_link(aes(alpha=weight)) +
  geom_node_point(shape="circle filled", fill="white") +
  geom_node_text(aes(label=name), hjust="outward", check_overlap=TRUE) +
  guides(edge_alpha=FALSE)



######### Ranking de importancia das palavras chave

# Podemos usar a relação força como uma forma de identificar termos importantes. 

# Utiliza-se a rede para classificar os termos de pesquisa por importância, com o objetivo 
# de eliminar alguns dos menos importantes. A "força" de cada termo na rede é o número de 
# outros termos com os quais ele aparece junto.

strengths <- strength(naivegraph)

# No topo estão os termos mais fracamente vinculados aos outros. Para alguns deles  
# pode-se comparar isso com as posições na visualização do gráfico, onde aparecem perto 
# das margens da figura.

data.frame(term=names(strengths), strength=strengths, row.names=NULL) %>%
  mutate(rank=rank(strength, ties.method="min")) %>%
  arrange(strength) ->
  term_strengths

term_strengths

# Plotagem da distribuição de "força" dos termos

# A figura mostra os termos em ordem crescente de "força" da esquerda para a direita. 
cutoff_fig <- ggplot(term_strengths, aes(x=rank, y=strength, label=term)) +
  geom_line() +
  geom_point() +
  geom_text(data=filter(term_strengths, rank>5), hjust="right", nudge_y=20, check_overlap=TRUE)

cutoff_fig


# Busca-se todos os termos acima de algum ponto de corte de importância e não deve-se 
# considerar os termos na na parte de baixo da distribuição.

# Uma maneira simples de decidir sobre um corte é escolher reter uma certa proporção da 
# "força" total da rede de termos de pesquisa, utilizando termos que representem uma 
# importância total de 80% na rede.

cutoff_cum <- find_cutoff(naivegraph, method="cumulative", percent=0.8)

cutoff_cum

# Plota a figura
cutoff_fig +
  geom_hline(yintercept=cutoff_cum, linetype="dashed")

# Uma vez encontrado um valor de corte, é necessário remover os termos com baixa 
# intensidade. 

get_keywords(reduce_graph(naivegraph, cutoff_cum))



######### Changepoints

# Existem certos pontos ao longo da classificação de termos onde a "força" do próximo termo 
# mais forte é muito maior do que a do anterior. Esses locais são chamados de pontos de 
# corte, uma vez que os termos abaixo deles têm "força" muito menor do que os que estão 
# acima.

cutoff_change <- find_cutoff(naivegraph, method="changepoint", knot_num=3)

cutoff_change

# Plota a figura
cutoff_fig +
  geom_hline(yintercept=cutoff_change, linetype="dashed")

In [ ]:
# Então, pode-se escolher um dos pontos de corte no vetor de termos.


g_redux <- reduce_graph(naivegraph, cutoff_change[1])
selected_terms <- get_keywords(g_redux)

selected_terms



######### Agrupando termos

# Agora que existe uma lista revisada de termos de pesquisa a partir dos resultados 
# da pesquisa ingênua, é necessário transformá-los em uma nova busca utilizada para 
# obter mais artigos relevantes para o mesmo tópico.

# Existem métodos para agrupar redes automaticamente em clusters, mas nem sempre são 
# tão confiáveis. A documentação do litsearchr recomenda fazer esta etapa manualmente. 

# Analisando termos de pesquisa e os colocando em uma lista de vetores separados, 
# um para cada subtópico.


grouped_terms <-list(
  manipulator=selected_terms[c(5, 11, 16, 17, 19, 21, 22, 23, 25, 27)],
  underwater=selected_terms[c(10, 11, 13, 12, 15)],
  techniques=selected_terms[c(2, 3, 6, 9, 8)]
  manipulator=selected_terms[c(31, 32, 27, 37, 79, 95, 94)],
  underwater=selected_terms[c(28, 39)],
  techniques=selected_terms[c(4, 5, 14, 15, 16)]
)

grouped_terms

# A função write_search () pega nossa lista de termos de pesquisa agrupados e escreve o 
# texto de uma nova pesquisa. 
write_search(
  grouped_terms,
  languages="English",
  exactphrase=TRUE,
  stemming=FALSE,
  closure="left",
  writesearch=TRUE
)

## <span style="color:cyan">O ciclo OTIMIZADO</span>

In [ ]:
library(bibliometrix)
biblioshiny()

In [ ]:
########## Verificando o novo resultado

# Carregndo o resultado

# As buscas foram realizadas nas bases de dados Compendex, IEEE e Scopus
new_search_directory <- "/home/anderson/Documentos/MinicursoBili/bir-mini-method-bili/code/litsearchr/example"

# Importa os dados das três bases, formando um único dataset
new_results <-
  litsearchr::import_results(directory = new_search_directory, verbose = TRUE)

# Remove os trabalhos duplicados
new_results <-
  litsearchr::remove_duplicates(naiveimport, field = "title", method = "string_osa")

# Agora é necessário verificar se os novos resultados parecem ser relevantes para o 
# tópico escolhido.

# Primeiro verifica-se se todos os resultados da pesquisa ingênua estão na nova 
# pesquisa. Uma vez que a pesquisa ingênua foi conduzida usando os termos mais importantes 
# para o tópico, e como esses mesmos termos ou outros muito semelhantes foram 
# incluídos na nova pesquisa, deve-se obter os mesmos artigos de volta entre 
# os novos resultados.

naiveresults %>%
  mutate(in_new_results=title %in% new_results[, "title"]) ->
  naiveresults

naiveresults %>%
  filter(!in_new_results) %>%
  select(title, keywords)


# Se começou-se a revisar a literatura sobre o tema escolhido, pode-se já saber os 
# títulos de alguns artigos importantes que foram escritos sobre o assunto. 
# Portanto, outra forma de verificar a nova pesquisa é verificar se ela inclui 
# resultados importantes específicos.

important_titles <- c(
  "Collision detection for underwater ROV manipulator systems"
)

data.frame(check_recall(important_titles, new_results[, "title"]))



In [ ]:
library(revtools)

#Mudar a pasta do arquivo 
data <- read_bibliography("~/Desktop/Document/revtools/scopusfinal_example.bib")
View(data) # Vê o dado previamente

#Quando terminar irá salvar um arquivo no formato .csv na raiz do sistema
result <- screen_abstracts(data)

#Lê os resultados
assigned.dat <- read_bibliography("~/Desktop/Document/revtools/revtools_filtred.csv")

View(assigned.dat)

# Seleciona os resultados e cria um arquivo no formato .bib na raiz do sistema
screened.dat.YES = subset(assigned.dat,  screened_abstracts =="selected")
write_bibliography(screened.dat.YES, "otimize_file.bib", format = "bib")

## <span style="color:cyan">O ciclo IMPACTO</span>

In [ ]:
library(bibliometrix)
biblioshiny()

## <span style="color:cyan">O ciclo PRODUÇÃO</span>

Leitura dos artigos
Análise e anotações no mendeley
Construção do mapa conceitual da pesquisa
Elaboração do documento do estudo do estado da arte

In [ ]:
# ggplot2 examples
library(ggplot2) 

In [ ]:
# create factors with value labels 
mtcars$gear <- factor(mtcars$gear,levels=c(3,4,5),
  	labels=c("3gears","4gears","5gears")) 
mtcars$am <- factor(mtcars$am,levels=c(0,1),
  	labels=c("Automatic","Manual")) 
mtcars$cyl <- factor(mtcars$cyl,levels=c(4,6,8),
   labels=c("4cyl","6cyl","8cyl")) 

In [ ]:
# Kernel density plots for mpg
# grouped by number of gears (indicated by color)
qplot(mpg, data=mtcars, geom="density", fill=gear, alpha=I(.5), 
   main="Distribution of Gas Milage", xlab="Miles Per Gallon", 
   ylab="Density")

In [ ]:
# Scatterplot of mpg vs. hp for each combination of gears and cylinders
# in each facet, transmittion type is represented by shape and color
qplot(hp, mpg, data=mtcars, shape=am, color=am, 
   facets=gear~cyl, size=I(3),
   xlab="Horsepower", ylab="Miles per Gallon") 

In [ ]:
# Separate regressions of mpg on weight for each number of cylinders
qplot(wt, mpg, data=mtcars, geom=c("point", "smooth"), 
   method="lm", formula=y~x, color=cyl, 
   main="Regression of MPG on Weight", 
   xlab="Weight", ylab="Miles per Gallon")

In [ ]:
# Boxplots of mpg by number of gears 
# observations (points) are overlayed and jittered
qplot(gear, mpg, data=mtcars, geom=c("boxplot", "jitter"), 
   fill=gear, main="Mileage by Gear Number",
   xlab="", ylab="Miles per Gallon")